In [1]:
import requests
from io import StringIO
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import datetime
import time

In [2]:
today = datetime.date.today()
day = today-datetime.timedelta(days=7)
print (str (day).replace("-", ""))

20210315


In [3]:
def check_symbol (stock_df ,label) :
    test_dr = pd.DataFrame()
    for i in range (len(stock_df)):
        if len(stock_df.iloc[i][label]) == 4:
            test_dr = test_dr.append(stock_df[i:i+1]).reset_index(drop=True)
    return test_dr

In [4]:
def extract(Stock_list, whole_df , label):
    start = 0
    end = 0
    point = Stock_list[0]
    for i in range (len(whole_df)):
        if whole_df.iloc[i][label] >= str(point):
            #print (whole_df.iloc[i][label] , i)
            if start != 0 :
                end = i
                break
            point = Stock_list[len(Stock_list)-1]
            start = i
    test =  pd.DataFrame(whole_df[start:end+1])
    return test

In [5]:
def find_index(Stock_list, extract, label,  null) :
    institutional = pd.DataFrame()
    for i in range (len(Stock_list)):
        a = extract[extract[label] == str(Stock_list[i])].index
        if len(a)== 1  :
            institutional=institutional.append(extract.iloc[a])
        elif len(a) == 0 :
            institutional=institutional.append(null)
    return institutional.reset_index(drop=True)

In [6]:
def devide(list1 , list2) :
    out_come = list()
    for i in range (len(list1)):
        if type (list1[i]) != float and  int(list2[i].replace(",", "")) >0 :
            x = format(int(list1[i].replace(",", "")) / int(list2[i].replace(",", "")) ,'.3f')
        else :
            x = 0
        out_come.append(x)
    return out_come

In [16]:
Stock_list = pd.read_csv('name.csv')["name"]
date_list = pd.read_csv('date.csv')['date'].tolist()
whole_data = pd.DataFrame()
Stock_index = pd.read_csv('Market_index.csv')

In [19]:
# generate date
i = 0 
today = datetime.date.today()
day = str (today-datetime.timedelta(days=i)).replace("-", "")
record_day =  max(date_list) 

sleep_download_time = 3

while int(day) > record_day  :
    # get basic data
    
    r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + day + '&type=ALL')
    if len(r.text) < 100 :
        time.sleep(sleep_download_time)
        i+=1
        day = str (today-datetime.timedelta(days=i)).replace("-", "")
        print (day ," is not work day!")
        continue
        
    label_ = "證券代號"
    df = pd.read_csv(StringIO(r.text.replace("=", "")),  header=["證券代號" in l for l in r.text.split("\n")].index(True)-1).drop(['Unnamed: 16'],axis = 1)
    df = df.dropna().reset_index(drop=True)
    time.sleep(sleep_download_time)
    
    # process the basic data by name list
    base_df = extract(Stock_list , df , label_).reset_index(drop=True)
    stock_df = find_index(Stock_list, base_df, label_,  df[-1:])
    
    ## get institution data
    r = requests.get('https://www.twse.com.tw/fund/T86?response=csv&date=' + day + '&selectType=ALL')
    r = r.text.replace("=", "")
    df2 = pd.read_csv(StringIO(r.replace("  \n", "")),  header=["證券代號" in l for l in r.split("\r\n")].index(True))
    df2.sort_values(by='證券代號' , ascending= True, axis =0 , inplace=True)
    df2 = df2.reset_index(drop=True).drop(['Unnamed: 19'],axis = 1)
    
    institutional = extract(Stock_list , df2 , label_).reset_index(drop=True)
    institutional = find_index(Stock_list, institutional, label_,  df2[-1:])
    time.sleep(sleep_download_time)
    ## process margin data
    r = requests.get('https://www.twse.com.tw/exchangeReport/MI_MARGN?response=csv&date=' + day + '&selectType=ALL')
    r = r.text.replace("=", "")
    df3 = pd.read_csv(StringIO(r), header=["股票代號" in l for l in r.split("\n")].index(True)).drop(['Unnamed: 16'],axis = 1)

    label_ = "股票代號"
    margin = extract(Stock_list, df3 , label_).reset_index(drop=True)
    margin = find_index(Stock_list, margin, label_,  df3[-1:])
    ratio = devide(margin['今日餘額.1'], margin['今日餘額'] )
    margin ['ratio'] = ratio 
    time.sleep(sleep_download_time)
    
    # merge to basic df
    stock_df['融資買'] = margin['買進']
    stock_df['融資賣'] = margin['賣出']
    stock_df['融資餘'] = margin['今日餘額']
    stock_df['融卷買'] = margin['買進.1']
    stock_df['融卷賣'] = margin['賣出.1']
    stock_df['融卷餘'] = margin['今日餘額.1']
    stock_df['ratio'] = margin ['ratio']

    stock_df['外資進出'] = institutional['外陸資買賣超股數(不含外資自營商)']
    stock_df['外資進'] = institutional['外陸資買進股數(不含外資自營商)']
    stock_df['外資出'] = institutional['外陸資賣出股數(不含外資自營商)']
    stock_df['投信進出'] = institutional['投信買賣超股數']
    stock_df['投信進'] = institutional['投信買進股數']
    stock_df['投信出'] = institutional['投信賣出股數']
    stock_df['自營進出'] = institutional['自營商買賣超股數(自行買賣)']
    stock_df['自營進'] = institutional['自營商買進股數(自行買賣)']
    stock_df['自營出'] = institutional['自營商賣出股數(自行買賣)']
    stock_df['避險進出'] = institutional['自營商買賣超股數(避險)']
    stock_df['避險進'] = institutional['自營商買進股數(避險)']
    stock_df['避險出'] = institutional['自營商賣出股數(避險)']
    stock_df['三大'] = institutional['三大法人買賣超股數']
    
    #stock_df  = stock_df.dropna().reset_index(drop=True)
    
    ## add date label
    date = [day for i in range(len(stock_df))]
    stock_df ["date"] = date
    whole_data = whole_data.append(stock_df)
    date_list.append(int(day))
    
    # for market index
    index_set = ['發行量加權股價指數' , '未含電子指數' , '電子類兩倍槓桿指數' ]
    r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date='+ day +'&type=IND')
    df = pd.read_csv(StringIO(r.text.replace("價格指數(臺灣證券交易所)\"\r\n", "")),  header=["指數" in l for l in r.text.split("\r\n")].index(True)).drop(['漲跌(+/-)','特殊處理註記','Unnamed: 6'],axis = 1)
    df1 = df.dropna()
    df1.columns = ['Index','收盤','漲跌','漲跌%']
    index = pd.DataFrame()
    for x in index_set :
        index = index.append(df1[df1.Index == x])
    index['date'] = [int(day) for x in range (len(index))]
    Stock_index = Stock_index.append(index)
    time.sleep(sleep_download_time)    
    print (day)
    
    i+=1
    day = str (today-datetime.timedelta(days=i)).replace("-", "")

20210322


In [20]:
date = pd.DataFrame()
date['date'] = date_list
date = date.drop_duplicates()
date = date.sort_values(by= 'date',ascending=False)
date.to_csv('date.csv', index = False)

In [21]:
date

,date
362,20210322
0,20210319
1,20210318
2,20210317
3,20210316
...,...
357,20190926
358,20190925
359,20190924
360,20190923


In [22]:
drop = whole_data.dropna().reset_index(drop=True)
whole_data = drop.drop(['漲跌(+/-)', '證券名稱' ,'最後揭示買價', '最後揭示買量', '最後揭示賣價', '最後揭示賣量'],axis = 1)

In [23]:
for l in whole_data.columns : 
    list_ = []
    for i in range(len(whole_data)) :
        k = whole_data[l][i]
        if type(k) == str :
            try :
                list_.append(float (k.replace(",", "")))
            except:
                print(l , i , k)
                list_.append(0)
                continue
        else : 
            list_.append(float(k))
    whole_data[l] = list_

In [24]:
Stock_index = Stock_index.drop_duplicates()
Stock_index = Stock_index.reset_index(drop=True)
Stock_index

,Index,收盤,漲跌,漲跌%,date
0,發行量加權股價指數,16211.7,264.85,1.66,20210303
1,未含電子指數,16115,273.49,1.73,20210303
2,電子類兩倍槓桿指數,40860.8,1287.77,3.25,20210303
3,發行量加權股價指數,15946.9,6.92,-0.04,20210302
4,未含電子指數,15841.5,60.53,-0.38,20210302
...,...,...,...,...,...
1084,未含電子指數,16330.2,191.04,-1.16,20210319
1085,電子類兩倍槓桿指數,39187.8,1153.94,-2.86,20210319
1086,發行量加權股價指數,"16,189.22",118.98,0.74,20210322
1087,未含電子指數,"16,498.68",168.52,1.03,20210322


In [25]:
for l in ['收盤','漲跌','漲跌%'] : 
    list_ = []
    for i in range(len(Stock_index)) :
        k = Stock_index[l][i]
        if type(k) == str :
            try :
                list_.append(float (k.replace(",", "")))
            except:
                print(l , i , k)
                list_.append(0)
                continue
        else : 
            list_.append(float(k))
    Stock_index[l] = list_

In [26]:
whole_data = whole_data [whole_data['開盤價'] > 0]
whole_data

,證券代號,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,...,投信進,投信出,自營進出,自營進,自營出,避險進出,避險進,避險出,三大,date
0,1102.0,16214783.0,7304.0,737947274.0,45.80,47.20,44.90,45.80,1.40,10.73,...,626000.0,35000.0,-675562.0,388439.0,1064001.0,21000.0,35000.0,14000.0,-4287562.0,20210322.0
1,1103.0,5272945.0,1322.0,98195711.0,18.70,18.85,18.45,18.65,0.50,35.19,...,0.0,0.0,100.0,100.0,0.0,0.0,0.0,0.0,-530900.0,20210322.0
2,1104.0,2145541.0,785.0,47574843.0,22.15,22.60,22.00,22.00,0.05,11.00,...,0.0,0.0,-1000.0,0.0,1000.0,0.0,0.0,0.0,91000.0,20210322.0
3,1108.0,1697202.0,605.0,21507950.0,12.85,12.85,12.55,12.65,0.00,13.75,...,0.0,0.0,-19000.0,0.0,19000.0,0.0,0.0,0.0,-525000.0,20210322.0
4,1109.0,1248655.0,459.0,26031480.0,20.75,21.00,20.75,20.85,0.35,7.81,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,143000.0,20210322.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,9944.0,204393.0,162.0,4197212.0,20.60,20.65,20.50,20.60,0.00,60.59,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-15000.0,20210322.0
895,9945.0,5743364.0,2889.0,255747699.0,44.60,44.90,44.15,44.85,0.30,9.71,...,0.0,0.0,3000.0,12000.0,9000.0,68000.0,68000.0,0.0,-912000.0,20210322.0
896,9946.0,1331748.0,530.0,22030060.0,16.55,16.65,16.45,16.65,0.10,29.21,...,0.0,0.0,-2000.0,0.0,2000.0,0.0,0.0,0.0,-119000.0,20210322.0
897,9955.0,235251.0,203.0,4401631.0,18.70,18.85,18.60,18.65,0.00,0.00,...,0.0,0.0,-7000.0,0.0,7000.0,0.0,0.0,0.0,11000.0,20210322.0


In [27]:
previous = pd.read_csv('clear.csv')
previous 

,證券代號,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,...,投信進,投信出,自營進出,自營進,自營出,避險進出,避險進,避險出,三大,date
0,1102.0,11384652.0,3431.0,503541629.0,43.95,44.40,43.80,44.40,0.05,10.40,...,1666921.0,235000.0,221687.0,231687.0,10000.0,-69000.0,0.0,69000.0,-2797153.0,20210319.0
1,1103.0,1133780.0,428.0,20350924.0,17.85,18.15,17.80,18.15,0.30,34.25,...,0.0,0.0,-1000.0,0.0,1000.0,0.0,0.0,0.0,-250000.0,20210319.0
2,1104.0,363683.0,260.0,7993054.0,22.10,22.30,21.85,21.95,0.15,10.98,...,0.0,0.0,-1000.0,0.0,1000.0,0.0,0.0,0.0,-239000.0,20210319.0
3,1108.0,4732122.0,1325.0,59606468.0,12.20,12.75,12.20,12.65,0.40,13.75,...,0.0,0.0,-10000.0,0.0,10000.0,0.0,0.0,0.0,-555000.0,20210319.0
4,1109.0,389250.0,211.0,7952627.0,20.50,20.55,20.25,20.50,0.00,7.68,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-131000.0,20210319.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312968,9943.0,190438.0,168.0,12812017.0,67.50,67.50,67.20,67.20,0.20,11.18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-52000.0,20190920.0
312969,9945.0,2752245.0,1096.0,116365981.0,42.60,42.60,42.10,42.10,0.50,4.38,...,0.0,81000.0,40000.0,40000.0,0.0,-56000.0,0.0,56000.0,-684265.0,20190920.0
312970,9946.0,1184358.0,469.0,27781296.0,23.50,23.60,23.30,23.60,0.10,5.58,...,0.0,0.0,-78000.0,12000.0,90000.0,0.0,0.0,0.0,107000.0,20190920.0
312971,9955.0,538156.0,334.0,10280286.0,18.80,19.20,18.80,19.15,0.35,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44000.0,20190920.0


In [28]:
K = whole_data.append(previous).reset_index(drop=True)
K

,證券代號,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,...,投信進,投信出,自營進出,自營進,自營出,避險進出,避險進,避險出,三大,date
0,1102.0,16214783.0,7304.0,737947274.0,45.80,47.20,44.90,45.80,1.40,10.73,...,626000.0,35000.0,-675562.0,388439.0,1064001.0,21000.0,35000.0,14000.0,-4287562.0,20210322.0
1,1103.0,5272945.0,1322.0,98195711.0,18.70,18.85,18.45,18.65,0.50,35.19,...,0.0,0.0,100.0,100.0,0.0,0.0,0.0,0.0,-530900.0,20210322.0
2,1104.0,2145541.0,785.0,47574843.0,22.15,22.60,22.00,22.00,0.05,11.00,...,0.0,0.0,-1000.0,0.0,1000.0,0.0,0.0,0.0,91000.0,20210322.0
3,1108.0,1697202.0,605.0,21507950.0,12.85,12.85,12.55,12.65,0.00,13.75,...,0.0,0.0,-19000.0,0.0,19000.0,0.0,0.0,0.0,-525000.0,20210322.0
4,1109.0,1248655.0,459.0,26031480.0,20.75,21.00,20.75,20.85,0.35,7.81,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,143000.0,20210322.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313867,9943.0,190438.0,168.0,12812017.0,67.50,67.50,67.20,67.20,0.20,11.18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-52000.0,20190920.0
313868,9945.0,2752245.0,1096.0,116365981.0,42.60,42.60,42.10,42.10,0.50,4.38,...,0.0,81000.0,40000.0,40000.0,0.0,-56000.0,0.0,56000.0,-684265.0,20190920.0
313869,9946.0,1184358.0,469.0,27781296.0,23.50,23.60,23.30,23.60,0.10,5.58,...,0.0,0.0,-78000.0,12000.0,90000.0,0.0,0.0,0.0,107000.0,20190920.0
313870,9955.0,538156.0,334.0,10280286.0,18.80,19.20,18.80,19.15,0.35,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44000.0,20190920.0


In [29]:
K.columns

Index(['證券代號', '成交股數', '成交筆數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差',
       '本益比', '融資買', '融資賣', '融資餘', '融卷買', '融卷賣', '融卷餘', 'ratio', '外資進出', '外資進',
       '外資出', '投信進出', '投信進', '投信出', '自營進出', '自營進', '自營出', '避險進出', '避險進', '避險出',
       '三大', 'date'],
      dtype='object')

In [30]:
K.to_csv('clear.csv', index = False)
Stock_index.to_csv('Market_index.csv', index = False)